In [1]:
import tensorflow as tf
import numpy as np

/Users/jeremy.zhang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### Sess

In [2]:
def add(n):
    r = tf.constant(0)
    for i in range(n):
        r = tf.add(r, 1)
    return r

In [3]:
with tf.Session() as sess:
    r = sess.run(add(tf.constant(3).eval()))
    print(r)

3


#### tf.convert_to_tensor

In [43]:
arr = np.random.uniform(size=[2, 4])
print(arr)

arr_tensor = tf.convert_to_tensor(arr)

with tf.Session() as sess:
    print(" ")
    print(sess.run(arr_tensor))

[[0.48647716 0.4162351  0.1291369  0.95566755]
 [0.21917876 0.535551   0.46342173 0.495829  ]]
 
[[0.48647716 0.4162351  0.1291369  0.95566755]
 [0.21917876 0.535551   0.46342173 0.495829  ]]


#### tf.where

In [79]:
t = tf.constant([[1, 1, 1], [2, 1, 0]])

r = tf.where(tf.equal(t, 1))

with tf.Session() as sess:
    print(sess.run(t))
    print("----------")
    print(sess.run(r))

[[1 1 1]
 [2 1 0]]
----------
[[0 0]
 [0 1]
 [0 2]
 [1 1]]


#### tf.gather_nd

In [80]:
t = tf.constant([[1, 1, 1], [2, 1, 0]])

r = tf.gather_nd(t, tf.where(tf.equal(t, 1)))

with tf.Session() as sess:
    print(sess.run(t))
    print("----------")
    print(sess.run(r))

[[1 1 1]
 [2 1 0]]
----------
[1 1 1 1]


#### tf.cond

In [3]:
t1 = tf.constant(1)
t2 = tf.constant(2)

def f1(): return tf.add(t1, t2)
def f2(): return tf.subtract(t1, t2)

res = tf.cond(tf.less(t1, t2), f1, f2)

with tf.Session() as sess:
    r = sess.run(res)
    print(r)

3


#### tf.while_loop

In [6]:
t1 = tf.constant(1)
t2 = tf.constant(10)
t3 = tf.constant(2)

def cond(t1, t2, t3): 
    print("***")
    return tf.less(t1, t2)

def body(t1, t2, t3):
    t3 = tf.multiply(t3, 2)
    return [tf.add(t1, 1), t2, t3]

res = tf.while_loop(cond, body, [t1, t2, t3])

with tf.Session() as sess:
    print(sess.run(res))

***
[10, 10, 1024]


In [22]:
fix_iters = tf.constant(5)
t1 = tf.constant(1.2)
i = tf.constant(0)

def cond(t1, i, fix_iters):
    return tf.less(i, fix_iters)
    
def body(t1, i, fix_iters):
    t1 = tf.multiply(t1, 2.2)
    return [t1, tf.add(i, 1), fix_iters]

res = tf.while_loop(cond, body, [t1, i, fix_iters])

with tf.Session() as sess:
    print(sess.run(res)[0:2])

[61.843594, 5]


#### cond & loop

In [26]:
def cond_loop(t1, t2, iters):
    
    def cond(t1, t2, i):
        return tf.less(i, iters)
    
    def body(t1, t2, i):
        def increment(t1, t2):
            def f1(): return tf.add(t1, 1), tf.add(t2, 1)  # no parameters
            return f1

        def swap(t1, t2):
            def f2(): return t2, t1
            return f2
        
        t1, t2 = tf.cond(tf.less(i+1, iters), increment(t1, t2), swap(t1, t2))
        
        return [t1, t2, tf.add(i, 1)]
    
    return tf.while_loop(cond, body, [t1, t2, 0])

t1 = tf.constant(1)
t2 = tf.constant(5)
iters = tf.constant(3)

with tf.Session() as sess:
    loop = cond_loop(t1, t2, iters)
    print(sess.run(loop))

[7, 3, 3]


In [27]:
def cond_loop(t1, t2, iters):
    
    def cond(t1, t2, i):
        return tf.less(i, iters)
    
    def body(t1, t2, i):
        def increment(t1, t2):
            return tf.add(t1, 1), tf.add(t2, 1)  # no parameters
            
        def swap(t1, t2):
            return t2, t1
            
        t1, t2 = tf.cond(tf.less(i+1, iters), lambda: increment(t1, t2), lambda: swap(t1, t2))
        
        return [t1, t2, tf.add(i, 1)]
    
    return tf.while_loop(cond, body, [t1, t2, 0])

t1 = tf.constant(1)
t2 = tf.constant(5)
iters = tf.constant(3)

with tf.Session() as sess:
    loop = cond_loop(t1, t2, iters)
    print(sess.run(loop))

[7, 3, 3]


#### tf.scan
---
**Scan on the list of tensors unpacked from elems on dimension 0.**

The callable fn takes two tensors as arguments. The first argument is the accumulated value computed from the preceding invocation of fn. If initializer is None, elems must contain at least one element, and its first element is used as the initializer.

In [31]:
def fn(a, b): return tf.add(a, b)

elem = tf.constant([1, 2, 3, 4])
res = tf.scan(fn, elem, initializer=tf.constant(0))

with tf.Session() as sess:
    print(sess.run(res))

[ 1  3  6 10]


In [39]:
def fn(a, b): return tf.add(a, b)

elem = tf.constant([[1, 1, 2], [1, 2, 3]])
res = tf.scan(fn, elem, initializer=tf.constant([0, 0, 0]))

with tf.Session() as sess:
    print(elem.eval())
    print("-----")
    print(sess.run(res))

[[1 1 2]
 [1 2 3]]
-----
[[1 1 2]
 [2 3 5]]


In [36]:
def fn(a, b): return tf.add(a, b)

elem = tf.constant([[1, 1, 2], [1, 2, 3]])
res = tf.scan(fn, elem, initializer=tf.constant(0))

with tf.Session() as sess:
    print(elem.eval())
    print("-----")
    print(sess.run(res))

[[1 1 2]
 [1 2 3]]
-----
[[1 1 2]
 [2 3 5]]


#### tf.map_fn

In [45]:
def fn(x): return 2*x

elem = tf.constant([[1, 1, 2], [1, 2, 3]])
res = tf.map_fn(fn, elem)

l = elem.shape[1]+2

with tf.Session() as sess:
    print(elem.eval())
    print("-"*2*l)
    print(sess.run(res))

[[1 1 2]
 [1 2 3]]
----------
[[2 2 4]
 [2 4 6]]


#### tf.case
---
The pred_fn_pairs parameter is a **dict or list of pairs of size N**. Each pair contains a boolean scalar tensor and a python callable that creates the tensors to be returned if the boolean evaluates to True. default is a callable generating a list of tensors.

If `exclusive==True`, all predicates are evaluated.

If `exclusive==False`, execution stops at the first predicate which evaluates to True.

In [50]:
def f1(): return tf.constant(17)
def f2(): return tf.constant(23)
def f3(): return tf.constant(-1)

x = tf.constant(12)
y = tf.constant(20)

r = tf.case({tf.less(x, y): f1, tf.greater(x, y): f2},
         default=f3, exclusive=True)

with tf.Session() as sess:
    print(sess.run(r))

17


In [58]:
def f1(x, y): return tf.add(x, y)
def f2(x, y): return tf.subtract(x, y)
def f3(): return tf.constant(-1)

x = tf.constant(12)
y = tf.constant(20)

r = tf.case({tf.less(x, y): lambda: f1(x, y), tf.greater(x, y): lambda: f2(x, y)}, default=f3, exclusive=True)
# callable error
with tf.Session() as sess:
    print(sess.run(r))

32


#### tf.math.logical

In [64]:
r = tf.math.logical_and(tf.constant(True), tf.constant(True))
r1 =  tf.math.logical_and(tf.constant(True), tf.constant(False))

with tf.Session() as sess:
    print(sess.run(r))
    print(sess.run(r1))

True
False


#### tf.tile

In [91]:
r = tf.tile([3.2], [3])

with tf.Session() as sess:
    print(sess.run(r))

[3.2 3.2 3.2]


#### tf.scatter_update

In [61]:
ref = tf.Variable([1, 2, 3, 4, 5, 6, 7, 8])
indices = tf.constant([[4], [3], [1] ,[7]])
updates = tf.constant([9, 10, 11, 12])

update = tf.scatter_update(ref, tf.squeeze(indices), updates)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())  # initialize variable
    print(sess.run(update))

[ 1 11  3 10  9  6  7 12]


#### tf.scatter_nd_update

In [57]:
ref = tf.Variable([1, 2, 3, 4, 5, 6, 7, 8])
indices = tf.constant([[4], [3], [1] ,[7]])
updates = tf.constant([9, 10, 11, 12])
update = tf.scatter_nd_update(ref, indices, updates)

with tf.Session() as sess:
    print("indices", sess.run(tf.shape(indices)))
    sess.run(tf.global_variables_initializer())  # initialize variable
    print(sess.run(update))

indices [4 1]
[ 1 11  3 10  9  6  7 12]


In [89]:
ref = tf.Variable(np.zeros(shape=[6, 6], dtype=np.float32))
indices = tf.constant([[4, 4], [3, 0], [1, 5], [5, 0]], dtype=tf.int32)

updates = tf.constant([9, 10, 11, 12], dtype=tf.float32)
update = tf.scatter_nd_update(ref, indices, updates)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())  # initialize variable
    print(sess.run(update))

[[ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 11.]
 [ 0.  0.  0.  0.  0.  0.]
 [10.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  9.  0.]
 [12.  0.  0.  0.  0.  0.]]


#### tf.split

In [85]:
arr_tensor = tf.constant([[1, 1, 1, 1, 1], [2, 2, 2, 2, 2]])

r1, r2 = tf.split(arr_tensor, [3, 2], 1)

with tf.Session() as sess:
    print(sess.run(r1))
    print(" ")
    print(sess.run(r2))

[[1 1 1]
 [2 2 2]]
 
[[1 1]
 [2 2]]


#### tf update Varibale

In [1]:
n = 4

def cond(i): return tf.less_equal(i, n)
def body(i): 
    m = tf.get_variable("m", shape=[6], initializer=tf.constant_initializer(0))
    op = tf.scatter_update(m, i, 333)  # this is only an operation
    
    tf.get_variable_scope().reuse_variables()
    with tf.control_dependencies([op]):
        return tf.add(i, 1)

r = tf.while_loop(cond, body, [1])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(r))
    
    print(sess.run(tf.get_variable("m")))

/Users/jeremy.zhang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


5
[  0. 333. 333. 333. 333.   0.]


In [3]:
t = tf.get_variable("t", shape=[6], initializer=tf.constant_initializer(0))

tf.get_variable_scope().reuse_variables()
t2 = tf.get_variable("t")

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(t))
    print(t.name)
    print(sess.run(t2))
    print(t2.name)

[0. 0. 0. 0. 0. 0.]
t:0
[0. 0. 0. 0. 0. 0.]
t:0


#### tf.shape
- `tf.shape`
tf.shape is used for dynamic shape. If your tensor's shape is changable, use it. An example: a input is an image with changable width and height, we want resize it to half of its size, then we can write something like:
```new_height = tf.shape(image)[0] / 2```

- `tensor.get_shape`
tensor.get_shape is used for fixed shapes, which means the tensor's shape can be deduced in the graph.

In [2]:
m = tf.constant([[1, 2], [1, 1]])
var = tf.get_variable("var", shape=m.get_shape().as_list(), initializer=tf.constant_initializer(0))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("tf.shape:", sess.run(tf.shape(m)))
    print("get_shape:", m.get_shape())
    print(sess.run(var))

tf.shape: [2 2]
get_shape: (2, 2)
[[0. 0.]
 [0. 0.]]


#### tf.stack

In [9]:
m = tf.zeros(shape=[100, 80])
mm = tf.stack([m, m], axis=2)

with tf.Session() as sess:
    print(sess.run(tf.shape(mm)))

[100  80   2]


#### tf.slice

In [16]:
m = tf.zeros(shape=[15, 42, 38, 5])

slice_m = tf.slice(m, [0, 0, 0, 0], [1, -1, -1, 1])
slice_m2 = tf.slice(m, [0, 0, 0, 1], [-1, -1, -1, 2])

split_m2_1, split_m2_2 = tf.split(slice_m2, [1, 1], 3)

with tf.Session() as sess:
    print(sess.run(tf.shape(slice_m)))
    print(sess.run(tf.shape(slice_m2)))
    print(sess.run(tf.shape(split_m2_1)))
    print(sess.run(tf.shape(split_m2_2)))

[ 1 42 38  1]
[15 42 38  2]
[15 42 38  1]
[15 42 38  1]


#### tf.conv2d

In [22]:
m = tf.zeros(shape=[5, 42, 38, 3])
slice_m = tf.slice(m, [0, 0, 0, 1], [-1, -1, -1, 1])

slice_m_transpose = tf.transpose(slice_m, [3, 1, 2, 0])

filter_m = tf.random.uniform(shape=[1, 1, 5, 1])

slice_smooth = tf.nn.conv2d(slice_m_transpose, filter_m, [1, 1, 1, 1], "SAME")

with tf.Session() as sess:
    print("slice shape", sess.run(tf.shape(slice_m)))
    print("slice transpose shape", sess.run(tf.shape(slice_m_transpose)))
    print("filter shape", sess.run(tf.shape(filter_m)))
    print("slice smoothed shape", sess.run(tf.shape(slice_smooth)))


slice shape [ 5 42 38  1]
slice transpose shape [ 1 42 38  5]
filter shape [1 1 5 1]
slice smoothed shape [ 1 42 38  1]


#### tf.math.segment*

In [45]:
segments = tf.constant([1, 2, 2, 2, 2])  # must in increasing order
m = tf.constant([0, 1, 2, 0, 3])

res = tf.math.segment_sum(data=m, segment_ids=segments)

with tf.Session() as sess:
    print(sess.run(res))

[0 0 6]


#### tf.math.unsorted_segment_sum

In [48]:
segments = tf.constant([0, 1, 2, 2, 1, 2]) 
m = tf.constant([0, 1, 2, 0, 3, 1])

res = tf.math.unsorted_segment_sum(data=m, segment_ids=segments, num_segments=3)

with tf.Session() as sess:
    print(sess.run(res))

[0 4 3]


#### tf.get_variable

In [72]:
m = tf.placeholder(shape=[3, 3], dtype=tf.int32)
with tf.variable_scope("foo2", reuse=tf.AUTO_REUSE):
    v = tf.get_variable("v", shape=m.shape, validate_shape=False)
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(v))

[[ 0.70206904 -0.68499374  0.40125299]
 [-0.8797443   0.17862082  0.16595149]
 [-0.7161479   0.49256063  0.92273355]]
